## Imports


In [2]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
#import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in


In [3]:
# Get today's date
today = datetime.now() - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=6)

def format_date(date):
    day = date.day
    # Months 3-7 (March–July) should be full; all others abbreviated with a period
    if date.month in (3, 4, 5, 6, 7):
        month_str = date.strftime("%B")      # e.g. "March", "April", ...
    else:
        month_str = date.strftime("%b") + "."  # e.g. "Jan.", "Feb.", "Aug.", ...

    return f"{month_str} {day}"


# Format both dates
week_ending = format_date(today)
week_starting = format_date(one_week_ago)

print(week_starting, week_ending)

Aug. 12 Aug. 18


In [28]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [29]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [30]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [31]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [32]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,August-6-2025,Single Family Residential,9440 NW 24th Pl,Pembroke Pines,FL,33024.0,610000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/9440-...,Beaches MLS,F10508639,N,Y,26.029414,-80.27252


In [33]:
print('Input county name:')
county = input()

Input county name:


In [34]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [35]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [36]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [37]:
df2 = df[(df['SOLD DATE'] >= one_week_ago) & (df['SOLD DATE'] < today)]

In [38]:
df2['SOLD DATE'].value_counts()

2025-08-15    18
2025-08-14     9
2025-08-13     6
2025-08-18     3
2025-08-16     1
Name: SOLD DATE, dtype: int64

In [39]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [40]:
# df2 = df2.drop(index=64)

In [41]:
df_top_ten = df2.head(10)

In [42]:
# df_top_ten = df2.head(10)

## Collect Agent Information


In [43]:
def process_response(url):
    try:
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"
        querystring = {"url": url}
        
        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }
        response = requests.get(base_url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()

        # Safely drill down, falling back to {} if any level is None or missing
        info = data.get('data') or {}
        panel = info.get('mainHouseInfoPanelInfo') or {}
        house = panel.get('mainHouseInfo') or {}

        # Extract listing agents
        listing_agents = house.get('listingAgents') or []
        list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName') if len(listing_agents) > 0 else None
        list_broker_name_1 = listing_agents[0].get('brokerName')           if len(listing_agents) > 0 else None
        list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName') if len(listing_agents) > 1 else None
        list_broker_name_2 = listing_agents[1].get('brokerName')           if len(listing_agents) > 1 else None

        # Extract buying agents
        buying_agents = house.get('buyingAgents') or []
        buy_agent_name_1  = buying_agents[0].get('agentInfo', {}).get('agentName') if len(buying_agents) > 0 else None
        buy_broker_name_1 = buying_agents[0].get('brokerName')             if len(buying_agents) > 0 else None
        buy_agent_name_2  = buying_agents[1].get('agentInfo', {}).get('agentName') if len(buying_agents) > 1 else None
        buy_broker_name_2 = buying_agents[1].get('brokerName')             if len(buying_agents) > 1 else None

        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return { key: None for key in [
            'url', 'list_agent_name_1','list_broker_name_1',
            'list_agent_name_2','list_broker_name_2',
            'buy_agent_name_1','buy_broker_name_1',
            'buy_agent_name_2','buy_broker_name_2'
        ] }

In [44]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [45]:
response_df = pd.DataFrame(response_list)

In [46]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [47]:
df_top_ten = merged_df

In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,...,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None
1,PAST SALE,2025-08-14,Condo/Co-op,4240 Galt Ocean Dr #1905,Fort Lauderdale,FL,33308.0,1467625.0,3.0,2.5,...,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Patricia Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc.",Shayna Lurie,"Compass Florida, LLC",David Marder,"Compass Florida, LLC"
2,PAST SALE,2025-08-15,Single Family Residential,740 NW 74th Ter,Plantation,FL,33317.0,975000.0,5.0,3.0,...,-80.247280,https://www.redfin.com/FL/Plantation/740-NW-74...,Ana Pflum,PFLUM Properties,None,None,None,Epic Choice Realty,None,None
3,PAST SALE,2025-08-15,Single Family Residential,1165 NW 132nd Ave,Pembroke Pines,FL,33028.0,875000.0,4.0,2.5,...,-80.326316,https://www.redfin.com/FL/Pembroke-Pines/1165-...,Lindsay Garcia,Redfin Corporation,None,None,Sonia Salazar,Rebonus Brokerage LLC,None,None
4,PAST SALE,2025-08-14,Single Family Residential,6041 NW 63rd Pl,Parkland,FL,33067.0,875000.0,5.0,3.0,...,-80.207368,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Kristine Doyle,Keller Williams Realty Consultants,None,None,Shelly Nelson,Partnership Realty Inc.,None,None
5,PAST SALE,2025-08-15,Single Family Residential,17001 NW 20th St,Pembroke Pines,FL,33028.0,799900.0,3.0,2.0,...,-80.374454,https://www.redfin.com/FL/Pembroke-Pines/17001...,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None
6,PAST SALE,2025-08-15,Multi-Family (2-4 Unit),1800 SW 97th Ave,Miramar,FL,33025.0,690000.0,9.0,6.0,...,-80.273882,https://www.redfin.com/FL/Miramar/1800-SW-97th...,Julia Pinzon,United Realty Group Inc,None,None,Gaelle Cazeau,World Class Realty,None,None
7,PAST SALE,2025-08-15,Single Family Residential,2960 NW 87th Ter,Coral Springs,FL,33065.0,660000.0,4.0,2.0,...,-80.241520,https://www.redfin.com/FL/Coral-Springs/2960-N...,David Whitelaw,Compass Florida LLC,Jaimi Sheldon,Compass Florida LLC,David Whitelaw,Compass Florida LLC,None,None
8,PAST SALE,2025-08-15,Single Family Residential,1248 NW 170th Ave,Pembroke Pines,FL,33028.0,650000.0,3.0,2.0,...,-80.373797,https://www.redfin.com/FL/Pembroke-Pines/1248-...,Tita Acosta,EXP Realty LLC,None,None,None,None,None,None
9,PAST SALE,2025-08-14,Single Family Residential,8802 NW 20th Mnr,Coral Springs,FL,33071.0,625000.0,4.0,2.0,...,-80.243356,https://www.redfin.com/FL/Coral-Springs/8802-N...,Andrew Mandel,EXP Realty LLC,None,None,None,Equity Real Property and Mgt.,None,None


## Current Week's Values


In [49]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

37


In [50]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$521,528


In [51]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$298


In [52]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$19,296,525


In [53]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0           37              521528          298                     19296525.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story


In [54]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [55]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [56]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
96
-----------
Input Previous Week Condo Average Sales Price:
302128
-----------
Input Previous Week Condo Average PSF:
253
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
29004305.0


In [57]:
# df_top_ten

## Correction section


In [58]:
# df.at[index#,'col_name']

## Format Data


In [59]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale


In [60]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter


In [61]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [62]:
pd.set_option('display.max_columns',None)

In [63]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange


In [64]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange
1,PAST SALE,2025-08-14,Condo/Co-op,4240 Galt Ocean Dr #1905,Fort Lauderdale,FL,33308.0,1467625.0,3.0,2.5,L'ambiance Beach,2318.0,NaN,2003.0,NaN,633.0,2373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10505659,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Patricia Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc.",Shayna Lurie,"Compass Florida, LLC",David Marder,"Compass Florida, LLC",2,blue
2,PAST SALE,2025-08-15,Single Family Residential,740 NW 74th Ter,Plantation,FL,33317.0,975000.0,5.0,3.0,Plantations Secluded Gardens,3061.0,16068.0,1970.0,NaN,319.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/740-NW-74...,Beaches MLS,F10507072,N,Y,26.132437,-80.247280,https://www.redfin.com/FL/Plantation/740-NW-74...,Ana Pflum,PFLUM Properties,None,None,None,Epic Choice Realty,None,None,3,blue
3,PAST SALE,2025-08-15,Single Family Residential,1165 NW 132nd Ave,Pembroke Pines,FL,33028.0,875000.0,4.0,2.5,PEMBROKE FALLS PHASE 2,2339.0,10077.0,1998.0,NaN,374.0,373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1165-...,MARMLS,A11833713,N,Y,26.016230,-80.326316,https://www.redfin.com/FL/Pembroke-Pines/1165-...,Lindsay Garcia,Redfin Corporation,None,None,Sonia Salazar,Rebonus Brokerage LLC,None,None,4,blue
4,PAST SALE,2025-08-14,Single Family Residential,6041 NW 63rd Pl,Parkland,FL,33067.0,875000.0,5.0,3.0,Countrys Point 119-29 B,2905.0,15479.0,1993.0,NaN,301.0,136.0,Sold,NaN,NaN,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Beaches MLS,F10515712,N,Y,26.309467,-80.207368,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Kristine Doyle,Keller Williams Realty Consultants,None,None,Shelly Nelson,Partnership Realty Inc.,None,None,5,blue
5,PAST SALE,2025-08-15,Single Family Residential,17001 NW 20th St,Pembroke Pines,FL,33028.0,799900.0,3.0,2.0,PEMBROKE ISLES 1,2076.0,6886.0,1996.0,NaN,385.0,395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/17001...,Stellar MLS as Distributed by MLS Grid,TB8402571,N,Y,26.024769,-80.374454,https://www.redfin.com/FL/Pembroke-Pines/17001...,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,6,blue
6,PAST SALE,2025-08-15,Multi-Family (2-4 Unit),1800 SW 97th Ave,Miramar,FL,33025.0,690000.0,9.0,6.0,EVERGLADES SUGAR & LAND C,2416.0,NaN,1973.0,NaN,286.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/1800-SW-97th...,MARMLS,A11824981,N,Y,25.992799,-80.273882,https://www.redfin.com/FL/Miramar/1800-SW-97th...,Julia Pinzon,United Realty Group Inc,None,None,Gaelle Cazeau,World Class Realty,None,None,7,blue
7,PAST SALE,2025-08-15,Single Family Residential,2960 NW 87th Ter,Coral Springs,FL,33065.0,660000.0,4.0,2.0,Forest Hills,2700.0,10059.0,1972.0,NaN,244.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Springs/2960-N...,Beaches MLS,RX-11112620,N,Y,26.266661,-80.241520,https://www.redfin.com/FL/Coral-Springs/2960-N...,David Whitelaw,Compass Florida LLC,Jaimi Sheldon,Compass Florida LLC,David Whitelaw,Compass Florida LLC,None,None,8,blue
8,PAST SALE,2025-08-15,Single Family Residential,1248 NW 170th Ave,Pembroke P

In [112]:
%store -r map_box_api_key

no stored variable or alias map_box_api_key


In [113]:
map_box_api_key = input("Enter Map Box API key")

In [114]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=9,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m


In [68]:
m.save('index.html')

NameError: name 'm' is not defined

## Data snagger


In [69]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [70]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange


In [71]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger


Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`


In [72]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_08182025


# CREATE TEMPLATE


In [73]:
muni_set = set(df_top_ten['CITY'])

In [74]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [75]:
df_top_ten.reset_index(inplace=True,drop=True)

In [76]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [77]:
top_sale

'Rio Vista Isles at 1505 SE 11th St in Fort Lauderdale'

In [78]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [79]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [80]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange,Rio Vista Isles at 1505 SE 11th St in Fort Lau...
1,PAST SALE,2025-08-14,Condo/Co-op,4240 Galt Ocean Dr #1905,Fort Lauderdale,FL,33308.0,1467625.0,3.0,2.5,L'ambiance Beach,2318.0,NaN,2003.0,NaN,633.0,2373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10505659,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Patricia Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc.",Shayna Lurie,"Compass Florida, LLC",David Marder,"Compass Florida, LLC",2,blue,L'ambiance Beach at 4240 Galt Ocean Dr #1905 i...
2,PAST SALE,2025-08-15,Single Family Residential,740 NW 74th Ter,Plantation,FL,33317.0,975000.0,5.0,3.0,Plantations Secluded Gardens,3061.0,16068.0,1970.0,NaN,319.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/740-NW-74...,Beaches MLS,F10507072,N,Y,26.132437,-80.247280,https://www.redfin.com/FL/Plantation/740-NW-74...,Ana Pflum,PFLUM Properties,None,None,None,Epic Choice Realty,None,None,3,blue,Plantations Secluded Gardens at 740 NW 74th Te...
3,PAST SALE,2025-08-15,Single Family Residential,1165 NW 132nd Ave,Pembroke Pines,FL,33028.0,875000.0,4.0,2.5,PEMBROKE FALLS PHASE 2,2339.0,10077.0,1998.0,NaN,374.0,373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1165-...,MARMLS,A11833713,N,Y,26.016230,-80.326316,https://www.redfin.com/FL/Pembroke-Pines/1165-...,Lindsay Garcia,Redfin Corporation,None,None,Sonia Salazar,Rebonus Brokerage LLC,None,None,4,blue,PEMBROKE FALLS PHASE 2 at 1165 NW 132nd Ave in...
4,PAST SALE,2025-08-14,Single Family Residential,6041 NW 63rd Pl,Parkland,FL,33067.0,875000.0,5.0,3.0,Countrys Point 119-29 B,2905.0,15479.0,1993.0,NaN,301.0,136.0,Sold,NaN,NaN,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Beaches MLS,F10515712,N,Y,26.309467,-80.207368,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Kristine Doyle,Keller Williams Realty Consultants,None,None,Shelly Nelson,Partnership Realty Inc.,None,None,5,blue,Countrys Point 119-29 B at 6041 NW 63rd Pl in ...
5,PAST SALE,2025-08-15,Single Family Residential,17001 NW 20th St,Pembroke Pines,FL,33028.0,799900.0,3.0,2.0,PEMBROKE ISLES 1,2076.0,6886.0,1996.0,NaN,385.0,395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/17001...,Stellar MLS as Distributed by MLS Grid,TB8402571,N,Y,26.024769,-80.374454,https://www.redfin.com/FL/Pembroke-Pines/17001...,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,6,blue,PEMBROKE ISLES 1 at 17001 NW 20th St in Pembro...
6,PAST SALE,2025-08-15,Multi-Family (2-4 Unit),1800 SW 97th Ave,Miramar,FL,33025.0,690000.0,9.0,6.0,EVERGLADES SUGAR & LAND C,2416.0,NaN,1973.0,NaN,286.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/1800-SW-97th...,MARMLS,A11824981,N,Y,25.992799,-80.273882,https://www.redfin.com/FL/Miramar/1800-SW-97th...,Julia Pinzon,United Realty Group Inc,None,None,Gaelle Cazeau,World Class Realty,None,None,7,blue,EVERGLADES SUGAR & LAND C at 1800 SW 97th Ave ...
7,PAST SALE,2025-08-15,Single Family Residential,2960 NW 87th Ter,Coral Springs,FL,33065.0,660000.0,4.0,2.0,Forest Hills,2700.0,10059.0,1972.0,NaN,244.0,NaN,

In [81]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Rio Vista Isles closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $625,000 to $2,425,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Miramar, Plantation, Coral Springs, Parkland, Pembroke Pines, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 37 condo sales totaling $19,296,525 million from Aug. 12 to Aug. 18. The previous week, brokers closed 96 condo sales totaling $29,004,305.

Last week’s units sold for an average of $521,528, higher than the $302,128 average price from the pre

In [82]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [83]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [84]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [85]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange,Rio Vista Isles at 1505 SE 11th St in Fort Lau...
1,PAST SALE,2025-08-14,Condo/Co-op,4240 Galt Ocean Dr #1905,Fort Lauderdale,FL,33308.0,1467625.0,3.0,2.5,L'ambiance Beach,2318.0,NaN,2003.0,NaN,633.0,2373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10505659,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Patricia Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc.",Shayna Lurie,"Compass Florida, LLC",David Marder,"Compass Florida, LLC",2,blue,L'ambiance Beach at 4240 Galt Ocean Dr #1905 i...
2,PAST SALE,2025-08-15,Single Family Residential,740 NW 74th Ter,Plantation,FL,33317.0,975000.0,5.0,3.0,Plantations Secluded Gardens,3061.0,16068.0,1970.0,NaN,319.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/740-NW-74...,Beaches MLS,F10507072,N,Y,26.132437,-80.247280,https://www.redfin.com/FL/Plantation/740-NW-74...,Ana Pflum,PFLUM Properties,None,None,None,Epic Choice Realty,None,None,3,blue,Plantations Secluded Gardens at 740 NW 74th Te...
3,PAST SALE,2025-08-15,Single Family Residential,1165 NW 132nd Ave,Pembroke Pines,FL,33028.0,875000.0,4.0,2.5,PEMBROKE FALLS PHASE 2,2339.0,10077.0,1998.0,NaN,374.0,373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1165-...,MARMLS,A11833713,N,Y,26.016230,-80.326316,https://www.redfin.com/FL/Pembroke-Pines/1165-...,Lindsay Garcia,Redfin Corporation,None,None,Sonia Salazar,Rebonus Brokerage LLC,None,None,4,blue,PEMBROKE FALLS PHASE 2 at 1165 NW 132nd Ave in...
4,PAST SALE,2025-08-14,Single Family Residential,6041 NW 63rd Pl,Parkland,FL,33067.0,875000.0,5.0,3.0,Countrys Point 119-29 B,2905.0,15479.0,1993.0,NaN,301.0,136.0,Sold,NaN,NaN,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Beaches MLS,F10515712,N,Y,26.309467,-80.207368,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Kristine Doyle,Keller Williams Realty Consultants,None,None,Shelly Nelson,Partnership Realty Inc.,None,None,5,blue,Countrys Point 119-29 B at 6041 NW 63rd Pl in ...
5,PAST SALE,2025-08-15,Single Family Residential,17001 NW 20th St,Pembroke Pines,FL,33028.0,799900.0,3.0,2.0,PEMBROKE ISLES 1,2076.0,6886.0,1996.0,NaN,385.0,395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/17001...,Stellar MLS as Distributed by MLS Grid,TB8402571,N,Y,26.024769,-80.374454,https://www.redfin.com/FL/Pembroke-Pines/17001...,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,6,blue,PEMBROKE ISLES 1 at 17001 NW 20th St in Pembro...
6,PAST SALE,2025-08-15,Multi-Family (2-4 Unit),1800 SW 97th Ave,Miramar,FL,33025.0,690000.0,9.0,6.0,EVERGLADES SUGAR & LAND C,2416.0,NaN,1973.0,NaN,286.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/1800-SW-97th...,MARMLS,A11824981,N,Y,25.992799,-80.273882,https://www.redfin.com/FL/Miramar/1800-SW-97th...,Julia Pinzon,United Realty Group Inc,None,None,Gaelle Cazeau,World Class Realty,None,None,7,blue,EVERGLADES SUGAR & LAND C at 1800 SW 97th Ave ...
7,PAST SALE,2025-08-15,Single Family Residential,2960 NW 87th Ter,Coral Springs,FL,33065.0,660000.0,4.0,2.0,Forest Hills,2700.0,10059.0,1972.0,NaN,244.0,NaN,

In [86]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Rio Vista Isles closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $625,000 to $2,425,000
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Miramar, Plantation, Coral Springs, Parkland, Pembroke Pines, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 37 condo sales totaling $19,296,525 million from Aug. 12 to Aug. 18. The previous week, brokers closed 96 condo sales totaling $29,004,305.

Last week’s units sold for an average of $521,528, higher than the $302,128 average price from the pre

In [87]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Fort-Lauderdale/1505-SE-11th-St-33316/home/41801639


In [88]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/4240-Galt-Ocean-Dr-33308/unit-1905/home/42003367


In [89]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Coral-Springs/8802-NW-20th-Mnr-33071/home/41436457


In [90]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Pembroke-Pines/1248-NW-170th-Ave-33028/home/41832392


In [91]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Fort-Lauderdale/1505-SE-11th-St-33316/home/41801639


In [92]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Coral-Springs/2960-NW-87th-Ter-33065/home/41427158


In [93]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-08-14,Single Family Residential,1505 SE 11th St,Fort Lauderdale,FL,33316.0,2425000.0,4.0,3.0,Rio Vista Isles,2732.0,NaN,1987.0,NaN,888.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Beaches MLS,F10520571,N,Y,26.109386,-80.127731,https://www.redfin.com/FL/Fort-Lauderdale/1505...,Tim Elmes,"Compass Florida, LLC",None,None,Patrick McTigue,American Marketing & Mgmnt.,None,None,1,orange,Rio Vista Isles at 1505 SE 11th St in Fort Lau...
1,PAST SALE,2025-08-14,Condo/Co-op,4240 Galt Ocean Dr #1905,Fort Lauderdale,FL,33308.0,1467625.0,3.0,2.5,L'ambiance Beach,2318.0,NaN,2003.0,NaN,633.0,2373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Beaches MLS,F10505659,N,Y,26.179471,-80.097189,https://www.redfin.com/FL/Fort-Lauderdale/4240...,Patricia Post,"Post and Post, Inc.",Debra Post PLLC,"Post and Post, Inc.",Shayna Lurie,"Compass Florida, LLC",David Marder,"Compass Florida, LLC",2,blue,L'ambiance Beach at 4240 Galt Ocean Dr #1905 i...
2,PAST SALE,2025-08-15,Single Family Residential,740 NW 74th Ter,Plantation,FL,33317.0,975000.0,5.0,3.0,Plantations Secluded Gardens,3061.0,16068.0,1970.0,NaN,319.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Plantation/740-NW-74...,Beaches MLS,F10507072,N,Y,26.132437,-80.247280,https://www.redfin.com/FL/Plantation/740-NW-74...,Ana Pflum,PFLUM Properties,None,None,None,Epic Choice Realty,None,None,3,blue,Plantations Secluded Gardens at 740 NW 74th Te...
3,PAST SALE,2025-08-15,Single Family Residential,1165 NW 132nd Ave,Pembroke Pines,FL,33028.0,875000.0,4.0,2.5,PEMBROKE FALLS PHASE 2,2339.0,10077.0,1998.0,NaN,374.0,373.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/1165-...,MARMLS,A11833713,N,Y,26.016230,-80.326316,https://www.redfin.com/FL/Pembroke-Pines/1165-...,Lindsay Garcia,Redfin Corporation,None,None,Sonia Salazar,Rebonus Brokerage LLC,None,None,4,blue,PEMBROKE FALLS PHASE 2 at 1165 NW 132nd Ave in...
4,PAST SALE,2025-08-14,Single Family Residential,6041 NW 63rd Pl,Parkland,FL,33067.0,875000.0,5.0,3.0,Countrys Point 119-29 B,2905.0,15479.0,1993.0,NaN,301.0,136.0,Sold,NaN,NaN,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Beaches MLS,F10515712,N,Y,26.309467,-80.207368,https://www.redfin.com/FL/Parkland/6041-NW-63r...,Kristine Doyle,Keller Williams Realty Consultants,None,None,Shelly Nelson,Partnership Realty Inc.,None,None,5,blue,Countrys Point 119-29 B at 6041 NW 63rd Pl in ...
5,PAST SALE,2025-08-15,Single Family Residential,17001 NW 20th St,Pembroke Pines,FL,33028.0,799900.0,3.0,2.0,PEMBROKE ISLES 1,2076.0,6886.0,1996.0,NaN,385.0,395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pembroke-Pines/17001...,Stellar MLS as Distributed by MLS Grid,TB8402571,N,Y,26.024769,-80.374454,https://www.redfin.com/FL/Pembroke-Pines/17001...,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,Jason McIntosh,RE/MAX REALTY UNLIMITED,None,None,6,blue,PEMBROKE ISLES 1 at 17001 NW 20th St in Pembro...
6,PAST SALE,2025-08-15,Multi-Family (2-4 Unit),1800 SW 97th Ave,Miramar,FL,33025.0,690000.0,9.0,6.0,EVERGLADES SUGAR & LAND C,2416.0,NaN,1973.0,NaN,286.0,NaN,Sold,NaN,NaN,https://www.redfin.com/FL/Miramar/1800-SW-97th...,MARMLS,A11824981,N,Y,25.992799,-80.273882,https://www.redfin.com/FL/Miramar/1800-SW-97th...,Julia Pinzon,United Realty Group Inc,None,None,Gaelle Cazeau,World Class Realty,None,None,7,blue,EVERGLADES SUGAR & LAND C at 1800 SW 97th Ave ...
7,PAST SALE,2025-08-15,Single Family Residential,2960 NW 87th Ter,Coral Springs,FL,33065.0,660000.0,4.0,2.0,Forest Hills,2700.0,10059.0,1972.0,NaN,244.0,NaN,

## Time on Market Calculator


In [94]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 6,11) ## List (Earlier) date
date2 = datetime(2025, 8, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

56


## Clean CSV for Datawrapper Chart


In [95]:
chart_df = df_top_ten

In [96]:
chart_df = chart_df.fillna(" ")

In [97]:
chart_df['LISTING AGENTS'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING AGENTS'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [98]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [99]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "BEDS", 
    "BATHS",
    "SOURCE", 
    "MLS#"
]

In [100]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [101]:
desired_order = [
    'SOLD DATE',
    'LOCATION',
    'ADDRESS',
    'PRICE',
    'SQUARE FEET',
    '$/SQUARE FEET',
    'YEAR BUILT',
    'LISTING AGENTS',
    'BUYING AGENTS'
]

chart_df = chart_df[desired_order]


In [102]:
chart_df = chart_df.rename(columns={'$/SQUARE FEET':'PPSF'})

In [103]:
# Coerce non-numeric to NaN, then you’ll have a float column
chart_df['PPSF'] = pd.to_numeric(chart_df['PPSF'], errors='coerce')

In [104]:
chart_df['SQUARE FEET'] = pd.to_numeric(chart_df['SQUARE FEET'], errors='coerce')

In [105]:
chart_df['PPSF'].astype(float)

0    888.0
1    633.0
2    319.0
3    374.0
4    301.0
5    385.0
6    286.0
7    244.0
8    398.0
9    302.0
Name: PPSF, dtype: float64

In [106]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['PPSF'] = chart_df['PPSF'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))

In [107]:
csv_date_string = today.strftime("%m_%d_%Y")

In [110]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [109]:
chart_df

,SOLD DATE,LOCATION,ADDRESS,PRICE,SQUARE FEET,PPSF,YEAR BUILT,LISTING AGENTS,BUYING AGENTS
0,2025-08-14,Rio Vista Isles,1505 SE 11th St in Fort Lauderdale,"$2,425,000","2,732",$888,1987,"Tim Elmes Compass Florida, LLC",Patrick McTigue American Marketing & Mgmnt.
1,2025-08-14,L'ambiance Beach,4240 Galt Ocean Dr #1905 in Fort Lauderdale,"$1,467,625","2,318",$633,2003,"Patricia Post Post and Post, Inc. Debra Post P...","Shayna Lurie Compass Florida, LLC David Marder..."
2,2025-08-15,Plantations Secluded Gardens,740 NW 74th Ter in Plantation,"$975,000","3,061",$319,1970,Ana Pflum PFLUM Properties,Epic Choice Realty
3,2025-08-15,PEMBROKE FALLS PHASE 2,1165 NW 132nd Ave in Pembroke Pines,"$875,000","2,339",$374,1998,Lindsay Garcia Redfin Corporation,Sonia Salazar Rebonus Brokerage LLC
4,2025-08-14,Countrys Point 119-29 B,6041 NW 63rd Pl in Parkland,"$875,000","2,905",$301,1993,Kristine Doyle Keller Williams Realty Consulta...,Shelly Nelson Partnership Realty Inc.
5,2025-08-15,PEMBROKE ISLES 1,17001 NW 20th St in Pembroke Pines,"$799,900","2,076",$385,1996,Jason McIntosh RE/MAX REALTY UNLIMITED,Jason McIntosh RE/MAX REALTY UNLIMITED
6,2025-08-15,EVERGLADES SUGAR & LAND C,1800 SW 97th Ave in Miramar,"$690,000","2,416",$286,1973,Julia Pinzon United Realty Group Inc,Gaelle Cazeau World Class Realty
7,2025-08-15,Forest Hills,2960 NW 87th Ter in Coral Springs,"$660,000","2,700",$244,1972,David Whitelaw Compass Florida LLC Jaimi Sheld...,David Whitelaw Compass Florida LLC
8,2025-08-15,LAKES OF WESTERN PINES RE,1248 NW 170th Ave in Pembroke Pines,"$650,000","1,633",$398,1997,Tita Acosta EXP Realty LLC,
9,2025-08-14,Ramblewood 76-49 B,8802 NW 20th Mnr in Coral Springs,"$625,000","2,071",$302,1975,Andrew Mandel EXP Realty LLC,Equity Real Property and Mgt.
